In [8]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [9]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/hannserr/schulich_data_science/main/non_profits.csv')

In [10]:
# Create a new model
model = gp.Model('NonprofitFunding')

total_budget = 50000000

# Create lists for alpha and beta values
alpha_value = df['alpha_i'].tolist()
beta_value = df['beta_i'].tolist()

# Number of nonprofits
num_nonprofits = len(df)

# Decision variables
allocation = model.addVars(num_nonprofits, lb=0, vtype=GRB.CONTINUOUS, name="allocation")
exponent_var = model.addVars(num_nonprofits, lb=0, vtype=GRB.CONTINUOUS, name="exponent")  

# Objective function
model.setObjective(gp.quicksum(3 * exponent_var[i] for i in range(num_nonprofits)), GRB.MAXIMIZE) 

# Constraints
# Budget Constraint
model.addConstr(gp.quicksum(allocation[i] for i in range(num_nonprofits)) <= total_budget, "Budget")

# Power constraint using addGenConstrPow()
for i in range(num_nonprofits):
    model.addGenConstrPow(allocation[i], exponent_var[i], 1.0/3.0, f"Power_Constraint_{i}")

# Optimize the model
model.optimize()

# Check the optimization status
if model.Status == GRB.OPTIMAL:
    print("Optimal solution found.")
    # Print the decision variables
    for v in model.getVars():
        print(f"{v.varName} = {v.x}")
    print(f"Objective Value: {model.ObjVal}")
elif model.Status == GRB.INFEASIBLE:
    print("Model is infeasible.")
    # Compute and print the IIS (Irreducible Inconsistent Subsystem)
    model.computeIIS()
    model.write("model.ilp")
    print("IIS written to file 'model.ilp'")
elif model.Status == GRB.UNBOUNDED:
    print("Model is unbounded.")
else:
    print(f"Optimization was stopped with status {model.Status}")

print(f"Number of constraints: {model.NumConstrs}") 
print(f"Number of decision variables: {model.NumVars}")

# Sum of decision variables in the optimal solution
sum_decision_variables = sum(v.x for v in model.getVars())
print(f"Sum of decision variables: {sum_decision_variables}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1 rows, 360 columns and 180 nonzeros
Model fingerprint: 0xb760b187
Model has 180 general constraints
Variable types: 360 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+07, 5e+07]
Found heuristic solution: objective -0.0000000
Presolve added 540 rows and 12960 columns
Presolve time: 0.07s
Presolved: 541 rows, 13320 columns, 39060 nonzeros
Presolved model has 180 SOS constraint(s)
Variable types: 13320 continuous, 0 integer (0 binary)

Root relaxation: objective 3.522487e+04, 2449 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wo